In [1]:
# default_exp capsAnalaysis

In [2]:
import sys

In [3]:
sys.path.append('/workspace/oct_ca_seg/oct/')

In [4]:
from oct.startup import *
from model import CapsNet
import numpy as np
import mlflow
from fastai.vision import *
import mlflow.pytorch as MLPY
from fastai.utils.mem import gpu_mem_get_all
from skimage import filters

In [5]:
gpu_mem_get_all()

[GPUMemory(total=16160, free=16148, used=12)]

In [6]:
cocodata_path = Path('/workspace/oct_ca_seg/COCOdata/')
train_path = cocodata_path/'train/images'
valid_path = cocodata_path/'valid/images'
test_path = cocodata_path/'test/images'

In [7]:
metrics = [sens, spec, dice, my_Dice_Loss, acc]

In [8]:
runsave_dir = Path('/workspace/oct_ca_seg/runsaves/')

In [9]:
run_name = 'DEEPCAP_09start_60e_001lr_fsmooth_deconv' #best
cfg_file = 'DEEPCAP_09start_60e_001lr_fsmooth_deconv'

In [10]:
run_dir = runsave_dir/run_name

In [11]:
def loadConfigRun(run_dir, name):
    with open(run_dir/name, 'r') as file:
        return json.load(file)

In [12]:
cfg_dict = loadConfigRun(run_dir, cfg_file)

In [13]:
cfg_dict

{'LEARNER': {'loss_alpha': 0.04,
  'loss_beta': 0.8,
  'bs': 24,
  'epochs': 60,
  'img_size': 256,
  'lr': 0.001,
  'num_workers': 1,
  'pct_start': 0.9,
  'runsave_dir': '/workspace/oct_ca_seg/runsaves/'},
 'MODEL': {'activation': 'relu',
  'batch_size': 2,
  'c_size': [256, 256],
  'cuda': True,
  'dataroot': '/workspace/oct_ca_seg/data_oct',
  'start_size': [256, 256],
  'device': 'cuda',
  'input_images': [0, 1, 2],
  'transforms': False,
  'dims1': 8,
  'dims2': 16,
  'dims3': 32,
  'epochs': 40,
  'f1dims': 8,
  'f1maps': 2,
  'f2dims': 16,
  'f2maps': 2,
  'inputchannels': 3,
  'normalization': 'batch',
  'primdims': 16,
  'primmaps': 4,
  'reconchannels': 1,
  'maps1': 4,
  'maps2': 8,
  'maps3': 16,
  'uptype': 'deconv',
  'up_type': 'deconv',
  'input_channels': 3},
 'DATSET': {'path': '/workspace/oct_ca_seg/COCOdata',
  'annotations_path': '/workspace/oct_ca_seg/COCOdata/train/images/medium_set_annotations.json'}}

In [14]:
config = DeepConfig(cfg_dict)

## Train, Valid

In [15]:
fn_get_y = lambda image_name: Path(image_name).parent.parent/('labels/'+Path(image_name).name)
codes = np.loadtxt(cocodata_path/'codes.txt', dtype=str)
tfms = get_transforms()
src = (SegCustomItemList
       .from_folder(cocodata_path, recurse=True, extensions='.jpg')
       .filter_by_func(lambda fname: Path(fname).parent.name == 'images', )
       .split_by_folder('train', 'valid')
       .label_from_func(fn_get_y, classes=codes))
src.transform(tfms, tfm_y=True, size=config.LEARNER.img_size)
data = src.databunch(cocodata_path,
                     bs=config.LEARNER.bs,
                     val_bs=2*config.LEARNER.bs,
                     num_workers = config.LEARNER.num_workers)
stats = [torch.tensor([0.2190, 0.1984, 0.1928]), torch.tensor([0.0645, 0.0473, 0.0434])]
data.normalize(stats);
data.c_in, data.c_out = 3, 2

In [16]:
ims = os.listdir(train_path, )

In [17]:
import PIL.Image as pilimage

In [20]:
from scipy.stats import mannwhitneyu as mw

In [2]:
import detectron2.model_zoo as d2

In [ ]:
from fastai